<a href="https://colab.research.google.com/github/gparin-07/Portfolio/blob/main/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch chatterbot
!pip install chatterbot_corpus
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer
from chatterbot.response_selection import get_random_response
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import random
import datetime

# ====================== PyTorch Intent Classifier ======================

class IntentClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(IntentClassifier, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_size, output_size)
        )

    def forward(self, x):
        return self.net(x)

# Expanded training data
intent_data = {
    "greetings": [
        "Hello", "Hi there", "Hey", "Hi", "Good morning", "Good afternoon",
        "Good evening", "Howdy", "Greetings", "Nice to meet you",
        "What's up?", "How's it going?"
    ],
    "goodbyes": [
        "Bye", "Goodbye", "See you later", "Take care", "Have a nice day",
        "Talk to you later", "Farewell", "Catch you later", "I'm leaving now",
        "See you soon"
    ],
    "thanks": [
        "Thank you", "Thanks", "Thanks a lot", "Thanks very much",
        "Thank you kindly", "Much appreciated", "I appreciate it",
        "You're the best", "Big thanks"
    ],
    "help": [
        "Can you help me?", "I need help", "Could you assist me?",
        "Help needed", "I need assistance", "Can you guide me?",
        "What should I do?", "Not sure what to do"
    ],
    "identity": [
        "Who are you?", "What are you?", "Tell me about yourself",
        "Are you human?", "Are you a robot?", "What do you do?"
    ],
    "jokes": [
        "Tell me a joke", "Make me laugh", "Say something funny",
        "Do you know any jokes?", "Can you joke?", "Entertain me"
    ],
    "time": [
        "What time is it?", "What's the time?", "Current time",
        "Could you tell me the time?", "Time now"
    ],
    "date": [
        "What's the date today?", "What date is it?", "Today's date",
        "Date now", "Current date", "What day is it?"
    ],
    "weather": [
        "What's the weather?", "How's the weather today?",
        "Is it raining?", "Should I take an umbrella?"
    ]
}

# Preprocess data
all_texts = [text.lower() for intent in intent_data.values() for text in intent]
all_labels = [intent for intent, texts in intent_data.items() for _ in texts]

# Vectorize text
vectorizer = TfidfVectorizer(max_features=500, stop_words='english')
X = vectorizer.fit_transform(all_texts).toarray()

# Encode labels
le = LabelEncoder()
y = le.fit_transform(all_labels)

# Convert to tensors
X_tensor = torch.FloatTensor(X)
y_tensor = torch.LongTensor(y)

# Initialize model
model = IntentClassifier(X.shape[1], 128, len(intent_data))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train model
for epoch in range(100):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_tensor)
    loss = criterion(outputs, y_tensor)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 20 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

def classify_intent(text):
    text = text.lower()
    features = vectorizer.transform([text]).toarray()
    with torch.no_grad():
        output = model(torch.FloatTensor(features))
        pred = torch.argmax(output).item()
    return le.inverse_transform([pred])[0]

# ====================== ChatterBot Integration ======================

chatbot = ChatBot(
    "PyBot",
    response_selection_method=get_random_response,
    logic_adapters=[
        {
            'import_path': 'chatterbot.logic.BestMatch',
            'default_response': "I'm not sure what you mean. Could you rephrase that?",
            'maximum_similarity_threshold': 0.90
        }
    ]
)

# Comprehensive training data
trainer = ListTrainer(chatbot)
training_data = [
    # Greetings
    "Hello", "Hi there! How can I help you today?",
    "Hey", "Hello! What would you like to know?",
    "Good morning", "Good morning! How are you today?",
    "Good afternoon", "Good afternoon! What can I do for you?",

    # Farewells
    "Bye", "Goodbye! Have a great day!",
    "See you later", "Take care! Come back soon!",

    # Thanks
    "Thank you", "You're welcome!",
    "Thanks a lot", "My pleasure!",

    # Help
    "Can you help me?", "Of course! What do you need help with?",
    "I need help", "I'm happy to help. Tell me what you need.",

    # Identity
    "Who are you?", "I'm PyBot, an AI chatbot designed to help you!",
    "What are you?", "I'm a conversational AI trained to chat with you!",

    # Jokes
    "Tell me a joke", "Why don't scientists trust atoms? Because they make up everything!",
    "Say something funny", "Why did the scarecrow win an award? Because he was outstanding in his field!",

    # Time
    "What time is it?", f"It's currently {datetime.datetime.now().strftime('%H:%M')}",

    # Date
    "What's the date today?", f"Today is {datetime.datetime.now().strftime('%B %d, %Y')}",

    # Weather
    "What's the weather?", "I can't check the weather, but you might want to look outside!",
    "Should I bring an umbrella?", "It's always good to be prepared, just in case!"
]

trainer.train(training_data)

# ====================== Chat Interface ======================

def get_response(input_text):
    try:
        # First try to classify the intent
        intent = classify_intent(input_text)

        # Special handling for some intents
        if intent == "time":
            return f"It's currently {datetime.datetime.now().strftime('%H:%M')}"
        elif intent == "date":
            return f"Today is {datetime.datetime.now().strftime('%B %d, %Y')}"
        elif intent == "jokes":
            jokes = [
                "Why don't eggs tell jokes? They'd crack each other up!",
                "I invented a new word: Plagiarism!",
                "What rock group has four men that don't sing? Mount Rushmore!"
            ]
            return random.choice(jokes)

        # Fall back to ChatterBot
        response = chatbot.get_response(input_text)
        return str(response)

    except Exception as e:
        return "I encountered an error. Could you try asking differently?"

print("""
==========================================
  PyBot - AI Chatbot (type 'quit' to exit)
==========================================
""")

while True:
    try:
        user_input = input("You: ")
        if user_input.lower() == 'quit':
            print("PyBot: Goodbye! Come back soon!")
            break

        response = get_response(user_input)
        print("PyBot:", response)

    except (KeyboardInterrupt, EOFError):
        print("\nPyBot: Goodbye!")
        break
